In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
# Load the dataset
data = pd.read_csv("/content/Assignment-1_Data.csv")

# Step 2: Explore the dataset
print("Number of rows:", len(data))
print("Columns:", data.columns)
print("Data types:\n", data.dtypes)
print("Missing values:\n", data.isnull().sum())

<ipython-input-2-6a2326c44bbe>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/Assignment-1_Data.csv")


Number of rows: 522064
Columns: Index(['BillNo', 'Itemname', 'Quantity', 'Date', 'Price', 'CustomerID',
       'Country'],
      dtype='object')
Data types:
 BillNo         object
Itemname       object
Quantity        int64
Date           object
Price         float64
CustomerID    float64
Country        object
dtype: object
Missing values:
 BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64


In [3]:
# Handle missing values
data.dropna(inplace=True)

# Remove duplicates
data.drop_duplicates(inplace=True)

In [4]:
# Perform data aggregation
transaction_data = data.groupby(['BillNo', 'CustomerID', 'Date'])['Itemname'].apply(list).reset_index(name='Items')


In [5]:
# Generate itemsets
te = TransactionEncoder()
itemsets = te.fit_transform(transaction_data['Items'])
itemsets = pd.DataFrame(itemsets, columns=te.columns_)

In [6]:
# Apply Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(itemsets, min_support=0.01, use_colnames=True)

In [7]:
# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [8]:
# Evaluate and interpret the rules
print("\nGenerated Association Rules:")
for index, rule in rules.iterrows():
    print("Antecedents:", list(rule.antecedents))
    print("Consequents:", list(rule.consequents))
    print("Support:", rule.support)
    print("Confidence:", rule.confidence)
    print("Lift:", rule.lift)
    print("====================================")



Generated Association Rules:
Antecedents: ['60 CAKE CASES DOLLY GIRL DESIGN']
Consequents: ['PACK OF 72 RETROSPOT CAKE CASES']
Support: 0.010058260965153347
Confidence: 0.5430267062314541
Lift: 9.95950392457165
Antecedents: ['60 TEATIME FAIRY CAKE CASES']
Consequents: ['PACK OF 72 RETROSPOT CAKE CASES']
Support: 0.01731340002198527
Confidence: 0.5
Lift: 9.170362903225806
Antecedents: ['ALARM CLOCK BAKELIKE CHOCOLATE']
Consequents: ['ALARM CLOCK BAKELIKE GREEN']
Support: 0.011377377157304606
Confidence: 0.6634615384615385
Lift: 15.45585048754063
Antecedents: ['ALARM CLOCK BAKELIKE CHOCOLATE']
Consequents: ['ALARM CLOCK BAKELIKE RED']
Support: 0.011981972078707267
Confidence: 0.6987179487179488
Lift: 14.868391063127905
Antecedents: ['ALARM CLOCK BAKELIKE IVORY']
Consequents: ['ALARM CLOCK BAKELIKE GREEN']
Support: 0.014949983511047598
Confidence: 0.5900216919739696
Lift: 13.745012373590784
Antecedents: ['ALARM CLOCK BAKELIKE ORANGE']
Consequents: ['ALARM CLOCK BAKELIKE GREEN']
Support: 

In [9]:
# Apply the MBA model
def get_recommendations(items):
    consequent_items = set(association_rules['consequents'].apply(lambda x: list(x)[0]))
    recommended_items = consequent_items.intersection(set(items))
    return recommended_items



In [10]:
# Example usage: Provide a set of items and get recommendations

# Define a set of items
item_set = {'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE PINK'}

# Filter the association rules based on the antecedent items
filtered_rules = rules[rules['antecedents'] == item_set]

# Sort the rules by confidence in descending order
filtered_rules = filtered_rules.sort_values(by='confidence', ascending=False)

# Get the recommended consequent items
recommendations = filtered_rules['consequents'].iloc[0]

print("Recommended Items:")
for item in recommendations:
    print(item)


Recommended Items:
ALARM CLOCK BAKELIKE RED


In [11]:
def get_recommendations(items, rules, min_confidence=0.5):
    recommendations = set()
    for index, rule in rules.iterrows():
        antecedents = set(rule.antecedents)
        consequents = set(rule.consequents)
        if items.issubset(antecedents) and rule.confidence >= min_confidence:
            recommendations.update(consequents)
    return recommendations

# Example usage
items = {'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE PINK'}
recommendations = get_recommendations(items, rules)
print("Recommended Items:", recommendations)


Recommended Items: {'ALARM CLOCK BAKELIKE RED'}


In [12]:
def get_recommendations(items, rules, min_confidence):
    recommended_items = set()
    for index, rule in rules.iterrows():
        antecedents = set(rule.antecedents)
        consequents = set(rule.consequents)
        if items.issubset(antecedents) and rule.confidence >= min_confidence:
            recommended_items.update(consequents)
    return recommended_items


In [13]:
# Define the list of item names
item_list = ['LUNCH BAG PINK POLKADOT', 'LUNCH BAG CARS BLUE', 'LUNCH BAG  BLACK SKULL', 'LUNCH BAG RED RETROSPOT', 'ROSES REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER', 'REGENCY CAKESTAND 3 TIER', 'GREEN REGENCY TEACUP AND SAUCER', 'LUNCH BAG SUKI DESIGN', 'LUNCH BAG WOODLAND', 'LUNCH BAG SPACEBOY DESIGN']

# Print the items with corresponding numbers
item_numbers = {}
print("Select the input items:")
for i, item in enumerate(item_list, start=1):
    item_numbers[i] = item
    print(f"{i}. {item}")

# Get the number of input items from the user
num_items = int(input("Enter the number of input items: "))

# Get the input items from the user by their numbers
input_items = set()
for i in range(num_items):
    item_number = int(input(f"Enter item {i+1} number: "))
    item_name = item_numbers.get(item_number)
    if item_name:
        input_items.add(item_name)
    else:
        print("Invalid item number. Please try again.")

min_confidence = 0.5
recommendations = get_recommendations(input_items, rules, min_confidence)
print("Recommended Items:", recommendations)

Select the input items:
1. LUNCH BAG PINK POLKADOT
2. LUNCH BAG CARS BLUE
3. LUNCH BAG  BLACK SKULL
4. LUNCH BAG RED RETROSPOT
5. ROSES REGENCY TEACUP AND SAUCER
6. PINK REGENCY TEACUP AND SAUCER
7. REGENCY CAKESTAND 3 TIER
8. GREEN REGENCY TEACUP AND SAUCER
9. LUNCH BAG SUKI DESIGN
10. LUNCH BAG WOODLAND
11. LUNCH BAG SPACEBOY DESIGN
Enter the number of input items: 2
Enter item 1 number: 1
Enter item 2 number: 2
Recommended Items: {'LUNCH BAG RED RETROSPOT', 'LUNCH BAG  BLACK SKULL.', 'LUNCH BAG SPACEBOY DESIGN'}


In [16]:
# Collect unique items from the "Itemname" column
item_list1 = data["Itemname"].unique().tolist()


# Print the items with corresponding numbers
item_numbers = {}
print("Select the input items:")
for i, item in enumerate(item_list1, start=1):
    item_numbers[i] = item
    print(f"{i}. {item}")

# Get the number of input items from the user
num_items = int(input("Enter the number of input items: "))

# Get the input items from the user by their numbers
input_items = set()
for i in range(num_items):
    item_number = int(input(f"Enter item {i+1} number: "))
    item_name = item_numbers.get(item_number)
    if item_name:
        input_items.add(item_name)
    else:
        print("Invalid item number. Please try again.")

min_confidence = 0.5
recommendations = get_recommendations(input_items, rules, min_confidence)
print("Recommended Items:", recommendations)

Select the input items:
1. WHITE HANGING HEART T-LIGHT HOLDER
2. WHITE METAL LANTERN
3. CREAM CUPID HEARTS COAT HANGER
4. KNITTED UNION FLAG HOT WATER BOTTLE
5. RED WOOLLY HOTTIE WHITE HEART.
6. SET 7 BABUSHKA NESTING BOXES
7. GLASS STAR FROSTED T-LIGHT HOLDER
8. HAND WARMER UNION JACK
9. HAND WARMER RED POLKA DOT
10. ASSORTED COLOUR BIRD ORNAMENT
11. POPPY'S PLAYHOUSE BEDROOM
12. POPPY'S PLAYHOUSE KITCHEN
13. FELTCRAFT PRINCESS CHARLOTTE DOLL
14. IVORY KNITTED MUG COSY
15. BOX OF 6 ASSORTED COLOUR TEASPOONS
16. BOX OF VINTAGE JIGSAW BLOCKS
17. BOX OF VINTAGE ALPHABET BLOCKS
18. HOME BUILDING BLOCK WORD
19. LOVE BUILDING BLOCK WORD
20. RECIPE BOX WITH METAL HEART
21. DOORMAT NEW ENGLAND
22. JAM MAKING SET WITH JARS
23. RED COAT RACK PARIS FASHION
24. YELLOW COAT RACK PARIS FASHION
25. BLUE COAT RACK PARIS FASHION
26. BATH BUILDING BLOCK WORD
27. ALARM CLOCK BAKELIKE PINK
28. ALARM CLOCK BAKELIKE RED
29. ALARM CLOCK BAKELIKE GREEN
30. PANDA AND BUNNIES STICKER SHEET
31. STARS GIFT TAPE


In [15]:
1

1